In [2]:
from spam_data.feature_engineering.base_builder import reshape_box_scores_to_matchups

In [3]:
df = reshape_box_scores_to_matchups()

Season 2021-22: 1230 games
Season 2022-23: 1230 games
Season 2023-24: 860 games
Season 2021-22: 1222 games
Season 2022-23: 1221 games
Season 2023-24: 851 games


C:\Users\james\Documents\Capstone\sports_betting\james\spam_data\spam_data\feature_engineering\utility_functions.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"])


In [4]:
df

,SEASON_ID,HOME_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NAME,GAME_ID,GAME_DATE,HOME_WL,HOME_MIN,HOME_PTS,HOME_FGM,...,AWAY_OREB,AWAY_DREB,AWAY_REB,AWAY_AST,AWAY_STL,AWAY_BLK,AWAY_TOV,AWAY_PF,GAME_RESULT,TOTAL_PTS
0,2021-22,1610612737,ATL,Atlanta Hawks,22101192,2022-04-06,W,242,118,43,...,6.0,38.0,44.0,26,4.0,4,14,17,1,221
1,2021-22,1610612737,ATL,Atlanta Hawks,22101163,2022-04-02,W,241,122,36,...,10.0,42.0,52.0,17,5.0,3,13,29,1,237
2,2021-22,1610612737,ATL,Atlanta Hawks,22100452,2022-03-31,W,239,131,45,...,15.0,32.0,47.0,19,6.0,5,14,20,1,238
3,2021-22,1610612737,ATL,Atlanta Hawks,22101102,2022-03-25,W,240,121,46,...,14.0,33.0,47.0,27,4.0,1,16,22,1,231
4,2021-22,1610612737,ATL,Atlanta Hawks,22101064,2022-03-20,L,240,112,41,...,18.0,36.0,54.0,22,12.0,1,15,23,0,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289,2023-24,1610612766,CHA,Charlotte Hornets,22300017,2023-11-14,L,240,105,36,...,10.0,32.0,42.0,26,12.0,5,7,19,0,216
3290,2023-24,1610612766,CHA,Charlotte Hornets,22300157,2023-11-08,L,240,116,41,...,13.0,27.0,40.0,26,10.0,6,10,26,0,248
3291,2023-24,1610612766,CHA,Charlotte Hornets,22300101,2023-10-30,L,240,121,47,...,10.0,36.0,46.0,25,4.0,8,13,19,0,254
3292,2023-24,1610612766,CHA,Charlotte Hornets,22300077,2023-10-27,L,240,99,33,...,12.0,41.0,53.0,28,7.0,6,23,29,0,210


In [26]:
df[["HOME_PTS", "AWAY_PTS", "GAME_RESULT"]]

,HOME_PTS,AWAY_PTS,GAME_RESULT
0,118,103,1
1,122,115,1
2,131,107,1
3,121,110,1
4,112,117,0
...,...,...,...
3289,105,111,0
3290,116,132,0
3291,121,133,0
3292,99,111,0


In [5]:
import numpy as np

In [20]:
def tester(row):
    teams = [row.HOME_TEAM_ID, row.AWAY_TEAM_ID]
    r = df[((df.HOME_TEAM_ID.isin(teams)) & (df.AWAY_TEAM_ID.isin(teams)) & (df.GAME_DATE < row.GAME_DATE))]
    r = r[r.GAME_DATE == r.GAME_DATE.max()]
    return r.TOTAL_PTS.item() if not r.empty else np.nan

In [37]:
def last_matchup_winner(row):
    teams = [row.HOME_TEAM_ID, row.AWAY_TEAM_ID]
    r = df[((df.HOME_TEAM_ID.isin(teams)) & (df.AWAY_TEAM_ID.isin(teams)) & (df.GAME_DATE < row.GAME_DATE))]
    r = r[r.GAME_DATE == r.GAME_DATE.max()]
    if r.empty:
        return np.nan
    winner = r.HOME_TEAM_ID.item() if r.GAME_RESULT.item() == 1 else r.AWAY_TEAM_ID.item()
    return 1 if row.HOME_TEAM_ID == winner else 0

In [21]:
df["LAST_MATCHUP_PTS"] = df.apply(tester, axis=1)

In [38]:
df["LAST_MATCHUP_WINNER"] = df.apply(last_matchup_winner, axis=1)

In [24]:
df.LAST_MATCHUP_PTS

0       231.0
1       218.0
2       240.0
3       240.0
4       201.0
        ...  
3289    211.0
3290    222.0
3291    188.0
3292    216.0
3293    282.0
Name: LAST_MATCHUP_PTS, Length: 3294, dtype: float64

In [39]:
df.LAST_MATCHUP_WINNER

0       1.0
1       0.0
2       1.0
3       0.0
4       1.0
       ... 
3289    1.0
3290    0.0
3291    0.0
3292    1.0
3293    1.0
Name: LAST_MATCHUP_WINNER, Length: 3294, dtype: float64

In [40]:
%load_ext autoreload
%autoreload 2

In [41]:
from spam_data.feature_engineering.last_matchup_feature_group import LastMatchupFeatureGroup

In [43]:
fg = LastMatchupFeatureGroup()

Season 2021-22: 1230 games
Season 2022-23: 1230 games
Season 2023-24: 860 games
Season 2021-22: 1222 games
Season 2022-23: 1221 games
Season 2023-24: 851 games


C:\Users\james\Documents\Capstone\sports_betting\james\spam_data\spam_data\feature_engineering\utility_functions.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"])


In [44]:
df = fg.process_data()

In [45]:
df.head()

,LAST_MATCHUP_PTS,LAST_MATCHUP_WINNER
GAME_ID,,
22101192,231,1
22101163,218,0
22100452,240,1
22101102,240,0
22101064,201,1


In [ ]:
df = 